In [7]:
import numpy as np
import pandas as p

#documents_categories = p.read_csv("/Users/astrachan/Desktop/outbrain/source_tables/documents_categories.csv")
#clicks_test = p.read_csv("/Users/astrachan/Desktop/outbrain/source_tables/clicks_test.csv")
#documents_meta = p.read_csv("/Users/astrachan/Desktop/outbrain/source_tables/documents_meta.csv")
#documents_entities = p.read_csv("/Users/astrachan/Desktop/outbrain/source_tables/documents_entities.csv")
#promoted_content = p.read_csv("/Users/astrachan/Desktop/outbrain/source_tables/promoted_content.csv", dtype = {"ad_id":int, "document_id":int, "campaign_id":int, "advertiser_id":int})
#sample_submission = p.read_csv("/Users/astrachan/Desktop/outbrain/source_tables/sample_submission.csv")
#documents_topics = p.read_csv("/Users/astrachan/Desktop/outbrain/source_tables/documents_topics.csv")
train = p.read_csv("/Users/astrachan/Desktop/outbrain/source_tables/clicks_train.csv", dtype={"display_id": int, "ad_id": int, "clicked" : bool})
events = p.read_csv("/Users/astrachan/Desktop/outbrain/source_tables/events.csv", dtype={"display_id": int, "uuid": str, "document_id" : int, "timestamp" : int, "platform" : str, "geo_location" : str})
#page_views = p.read_csv("/Users/astrachan/Desktop/outbrain/source_tables/page_views.csv")
#page_views_sample = p.read_csv("/Users/astrachan/Desktop/outbrain/source_tables/page_views_sample.csv")

In [8]:
#add number of ads on display
train_ad_count_per_display = train.groupby(['display_id'])['display_id'].agg({'ads_on_doc' : 'count'}).reset_index()
train = train.merge(train_ad_count_per_display, how = 'left', on = 'display_id')
del train_ad_count_per_display

In [9]:
#add the document_id describing ad
#add it later!!
#train = train.merge(promoted_content.drop(promoted_content.columns[[2,3]],axis = 1), how = 'left', on = 'ad_id')
#add the document_id of the display
events.drop(events.columns[[1,3,4,5]],axis = 1,inplace=True)
train = train.merge(events, how = 'left', on = 'display_id')

In [4]:
#split to validation and train, choosing display_id by random
display_ids = train.groupby(['display_id'])['display_id'].agg({'count' : 'count'}).reset_index().drop('count',axis = 1)
validation_displays = display_ids.sample(frac = 0.2)
train.reset_index(level=0, inplace=True)
validation = validation_displays.merge(train, how = 'inner', on = 'display_id')
train = train.drop(validation['index'])

In [10]:
ad_strength = train.groupby(['ad_id','ads_on_doc']).clicked.agg({'clicked' : 'sum', 'total' : 'count'}).reset_index()

In [11]:
#add expected chance of ad to be chosen [uni_chance]
ad_strength['uni_chance'] = 1 / ad_strength['ads_on_doc']
#add clicked precentage
ad_strength['clicked_percent'] = ad_strength['clicked'] / ad_strength['total']
#add normalized clicked precentage
ad_strength['clicked_percent_normalized'] = (ad_strength['clicked'] + 12 * ad_strength['clicked_percent'].mean()) / (12 + ad_strength['total'])
#create a likelihood column which shows how strong the actual 
#clicked percentage is in relation to the uniform chance, added regularization
#normalized likelihood
ad_strength['likelihood_normalized'] = ad_strength['clicked_percent_normalized'] / ad_strength['uni_chance']
#on the way to adding together same pairs with different ad_on_doc
ad_strength['like_mul_total_normalized'] = ad_strength['likelihood_normalized'] * ad_strength['total']

In [12]:
print ad_strength.head()

   ad_id  ads_on_doc  total  clicked  uni_chance  clicked_percent  \
0      1           3      1      0.0    0.333333         0.000000   
1      1           8      1      0.0    0.125000         0.000000   
2      2           4      3      0.0    0.250000         0.000000   
3      2           6     17      1.0    0.166667         0.058824   
4      2           8      1      0.0    0.125000         0.000000   

   clicked_percent_normalized  likelihood_normalized  \
0                    0.148420               0.445259   
1                    0.148420               1.187358   
2                    0.128630               0.514522   
3                    0.101016               0.606094   
4                    0.148420               1.187358   

   like_mul_total_normalized  
0                   0.445259  
1                   1.187358  
2                   1.543565  
3                  10.303604  
4                   1.187358  


In [13]:
#get scores for the ad_id grouped over the ad_on_doc
ad_strength = ad_strength.drop(ad_strength.columns[[1,3,4,5,6,7]],axis = 1).groupby(['ad_id']).sum().reset_index()
ad_strength['score'] = ad_strength['like_mul_total_normalized'] / ad_strength['total']
ad_strength.drop(ad_strength.columns[[1,2]],axis = 1, inplace=True)
ad_strength.sort_values('score',inplace=True, ascending=False)

In [14]:
print ad_strength.head()

         ad_id     score
32688    36041  5.879594
129111  144085  4.751383
228889  257136  4.688874
129109  144083  4.500740
1438      1615  4.275040


In [15]:
#create document_on_ad, intended calculating the statistical strenght of the ad document pair
document_on_ad = train.groupby(['document_id','ad_id','ads_on_doc']).clicked.agg({'clicked' : 'sum', 'total' : 'count'}).reset_index()

In [16]:
#add expected chance of ad to be chosen [uni_chance]
document_on_ad['uni_chance'] = 1 / document_on_ad['ads_on_doc']
#add clicked precentage
document_on_ad['clicked_percent'] = document_on_ad['clicked'] / document_on_ad['total']
#add normalized clicked precentage
document_on_ad['clicked_percent_normalized'] = (document_on_ad['clicked'] + 12 * document_on_ad['clicked_percent'].mean()) / (12 + document_on_ad['total'])
#create a likelihood column which shows how strong the actual 
#clicked percentage is in relation to the uniform chance, added regularization
#normalized likelihood
document_on_ad['likelihood_normalized'] = document_on_ad['clicked_percent_normalized'] / document_on_ad['uni_chance']
#on the way to adding together same pairs with different ad_on_doc
document_on_ad['like_mul_total_normalized'] = document_on_ad['likelihood_normalized'] * document_on_ad['total']

In [17]:
print document_on_ad.head()

   document_id   ad_id  ads_on_doc  total  clicked  uni_chance  \
0            2  207645           3      1      0.0    0.333333   
1            2  436985           3      1      0.0    0.333333   
2            2  439457           3      1      1.0    0.333333   
3            4   63334           9      1      0.0    0.111111   
4            4   68740           9      1      0.0    0.111111   

   clicked_percent  clicked_percent_normalized  likelihood_normalized  \
0              0.0                    0.167983               0.503950   
1              0.0                    0.167983               0.503950   
2              1.0                    0.244906               0.734719   
3              0.0                    0.167983               1.511849   
4              0.0                    0.167983               1.511849   

   like_mul_total_normalized  
0                   0.503950  
1                   0.503950  
2                   0.734719  
3                   1.511849  
4        

In [18]:
#get scores for the pairs ad_id doc_id grouped over the ad_on_doc (with mean on likelihood)
document_on_ad = document_on_ad.drop(document_on_ad.columns[[2,4,5,6,7,8]],axis = 1).groupby(['document_id','ad_id']).sum().reset_index()
document_on_ad['score'] = document_on_ad['like_mul_total_normalized'] / document_on_ad['total']
document_on_ad.drop(document_on_ad.columns[[2,3]],axis = 1, inplace=True)
document_on_ad.sort_values('score',inplace=True, ascending=False)

In [19]:
print document_on_ad.head()

          document_id   ad_id     score
8280589       1026091   36041  7.675055
7761034        961313  234107  7.334054
7763341        961361  132019  7.300757
21772541      2115556  167987  7.269273
21774448      2115593  167987  7.094973


In [20]:
document_on_ad.to_csv('/Users/astrachan/Desktop/outbrain/output_tables/document_on_ad.csv', index=False)
ad_strength.to_csv('/Users/astrachan/Desktop/outbrain/output_tables/ad_strength.csv', index = False)

In [21]:
#merge document on ad and ad_strength, can see the difference in the scores
scores = document_on_ad.merge(ad_strength, how = 'left', on = 'ad_id', suffixes = ('_doc_ad','_str'))

In [22]:
print scores.head(10)

   document_id   ad_id  score_doc_ad  score_str
0      1026091   36041      7.675055   5.879594
1       961313  234107      7.334054   1.233224
2       961361  132019      7.300757   1.625554
3      2115556  167987      7.269273   1.344240
4      2115593  167987      7.094973   1.344240
5      1976713   51402      7.094527   2.764124
6      2307478  148444      7.031208   4.177255
7      1035814  263141      6.963163   1.473062
8       373803   35316      6.844664   2.931826
9      1399897  132019      6.818983   1.625554


In [23]:
print validation.head()

   display_id     index   ad_id clicked  document_id
0    13587498  70121792     845   False       338866
1    13587498  70121793     852   False       338866
2    13587498  70121794  163624   False       338866
3    13587498  70121795  201025   False       338866
4    13587498  70121796  209912   False       338866


In [38]:
#take vali, validation without index and ads_on_doc
validation = validation.drop(validation.columns[[1,4]],axis = 1)
#merge vali with strength and document on ad
validation = validation.merge(document_on_ad, on= ['document_id','ad_id'], how = 'left')
validation = validation.merge(ad_strength, on = ['ad_id'], how = 'left', suffixes=('_adoc','_str'))

In [ ]:
print validation.head()

In [39]:
#check ammount of missing data and fill
print validation.shape
doc_ad_null = validation.score_adoc.isnull().sum()
str_null = validation.score_str.isnull().sum()
print 'ad_doc nulls: ' + str(doc_ad_null)
print 'portion: ' + repr(doc_ad_null / validation.shape[0])
print 'ad_strength nulls: ' + str(str_null)
print 'portion: ' + repr(str_null / validation.shape[0])

(17428626, 6)
ad_doc nulls: 5364252
portion: 0.30778398710259774
ad_strength nulls: 27083
portion: 0.0015539377573424318


In [ ]:
#for filling NAs, check median mean and min
print validation.score_adoc.mean()
print validation.score_adoc.median()
print validation.score_adoc.min()
print validation.score_str.mean()
print validation.score_str.median()
print validation.score_str.min()

In [ ]:
#fill NAs
validation.score_adoc = validation.score_adoc.fillna(validation.score_adoc.median())
validation.score_str = validation.score_str.fillna(validation.score_str.median())

In [43]:
#predict on validation
validation = validation.sort_values(['display_id','score_adoc','score_str'], ascending=False)

In [46]:
validation["seq"] = np.arange(validation.shape[0])
Y_seq = validation[validation.clicked == 1].seq.values
Y_first = validation[['display_id', 'seq']].drop_duplicates(subset='display_id', keep='first').seq.values
Y_ranks = Y_seq - Y_first
score = np.mean(1.0 / (1.0 + Y_ranks))
print("MAP: %.12f" % score)

MAP: 0.596897953235


In [50]:
max_rows = validation.groupby(['display_id'])['score_adoc'].transform(max) == validation['score_adoc']
final = validation[max_rows]
success = final[final['clicked'] == True]
float(len(success)) / float(len(final))

0.32520752044819357